# Data Preparation 

In this notebook, we inspect the data frames from different sources and prepare them for a unifying merge.

## General Setup

In [ ]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt

from signal_sigma.config.cfg_legacy import *

In [ ]:
# Begin of META era
START_DATE = "2012-12-30"

NB_NUMBER = 2

## Stocks from `yfinance`

In [ ]:
dfs_0 = {}

for ticker in STOCK_TICKERS:
    csvname = ticker.lower() + ".csv"
    csvpath_rel = os.path.join(DATA_STOCKS_DIR_RELPATH, csvname)
    dfs_0[ticker] = load_df_from_csv(csvpath_rel, NB_NUMBER)

In [ ]:
dfs_0["AMZN"].info()

### Stock Data Frame Processing

- Remove the null / header
- Make some features Engineering
- Change the column name
- Change the time type

In [ ]:
dfs = {}

for ticker in STOCK_TICKERS:
    df = dfs_0[ticker]

    # Remove rows containing any missing values.
    df = df.dropna()

    # Calculate new features based on price data

    # Calculate the difference between the high and low price for each day.
    df["delta_price"] = df["high"] - df["low"]
    # Calculate the average of the close, high, low, and open prices
    # as a metric for the day's price distribution.
    # WARNING: This might not be a standard financial metric. In consequence, 
    # this is also true for the following derived metrics.
    df["avg_price"] = (df["close"] + df["high"] + df["low"] + df["open"]) / 4
    # Calculate the ratio of the delta price to the average price.
    df["price_ratio"] = df["delta_price"] / df["avg_price"]
    # Calculate the difference between the trading volume and the average price
    df["invest"] = df["volume"] * df["avg_price"]

    # Other alternatives
    # XXX: Uncomment when time is ripe.

    # # Median Price (avoids extreme fluctuations)
    # df["median_price"] = (df["high"] + df["low"]) / 2

    # # Adjusted OHLC Mean (weights closing price higher)
    # df["adj_avg_price"] = (df["close"] * 2 + df["high"] + df["low"] + df["open"]) / 5

    # # Typical Price (common in financial indicators)
    # df["typical_price"] = (df["high"] + df["low"] + df["close"]) / 3

    # # Weighted Price (emphasizes high price more)
    # df["weighted_price"] = (df["high"] * 0.5 + df["low"] * 0.25 + df["close"] * 0.25)

    # # Volume-Weighted Average Price (VWAP)
    # df["vwap"] = (df["close"] * df["volume"]).cumsum() / df["volume"].cumsum()

    # # Exponential Moving Average (EMA-based mean for responsiveness)
    # df["ema_avg_price"] = df["close"].ewm(span=10, adjust=False).mean()

    # Include ticker in column names (with exception of date)
    df = df.rename(
        columns={
            col: f"{col}_{ticker}" for col in df.columns if col != "date"
        },
    )

    dfs[ticker] = df

###  Find the Max and Min of the Data column in each companies stock Data Frame

In [ ]:
dct_summary = {}

for ticker in STOCK_TICKERS:
    df = dfs[ticker]
    dr = {}
    dr["min_date"] = df["date"].min()
    dr["max_date"] = df["date"].max()
    dr["duration"] = dr["max_date"] - dr["min_date"]
    dct_summary[ticker] = dr

# Create a Pandas DataFrame to display the results
df_summary = pd.DataFrame.from_dict(dct_summary, orient="index")
df_summary.index.name = "Stock"

print("\nSummary: Stock Ranges\n")
print(df_summary.to_markdown(index=False))

In above result , It seems that the META is started from 2012 while almost the others started from 2000.

In [ ]:
# Store the cleaned dfs to disk
for ticker in STOCK_TICKERS:
    df = dfs[ticker]
    csvpath_rel = os.path.join(DATA_STOCKS_DIR_RELPATH, ticker.lower() + ".csv")
    store_df_as_csv(df, csvpath_rel, NB_NUMBER)

## Market Indices/Indexoids from `yfinance`

- Indices
- Commodities
- Sector ETFs (Proxies)
- Other Market Metrics

In [ ]:
csvpath_rel = DATA_YF_MIF_RELPATH
df_idxs_0 = load_df_from_csv(csvpath_rel, NB_NUMBER)

In [ ]:
df_idxs_0.isnull().sum()

The data frame has some missing values that need to be checked.

Let us restrict our analysis to the META era (after the 2012-05-31).

In [ ]:
df_idxs_1 = df_idxs_0[df_idxs_0["date"] > START_DATE]
df_idxs_1.isnull().sum()

In [ ]:
# Visualisation of missing values for the market indices

# plt.figure(figsize=(10, 6))
msno.matrix(df_idxs_1)
plt.title(f"Missing Value Matrix -- Macros")

plt.savefig(os.path.join(PLOTS_PATH, "missing-value-matrix.png"))
plt.show()

In [ ]:
df_idxs_2 = df_idxs_1.dropna()
df_idxs_2.isnull().sum()

In [ ]:
# Store the cleaned df to disk.
csvpath_rel = DATA_YF_MIF_RELPATH
store_df_as_csv(df_idxs_2, csvpath_rel, NB_NUMBER)

## Market Indicators from `fred`

In [ ]:
csvpath_rel = DATA_FED_CEI_RELPATH
df_inds_0 = load_df_from_csv(csvpath_rel, NB_NUMBER)

In [ ]:
df_inds_0.isnull().sum()

In [ ]:
df_inds_1 = df_inds_0[df_inds_0["date"] > START_DATE]
df_inds_1.isnull().sum()

In [ ]:
# Visualisation of missing values for the market indicators

msno.matrix(df_inds_1)
plt.title(f"Missing Value Matrix - FED")
plt.show()

In [ ]:
cols = [col for col in df_inds_1.columns if col not in ["gdp"]]

df_inds_2 = df_inds_1[cols].dropna()
df_inds_2.isnull().sum()

In [ ]:
# Store the cleaned df to disk.
csvpath_rel = DATA_FED_CEI_RELPATH
store_df_as_csv(df_inds_2, csvpath_rel, NB_NUMBER)